## Before running, you must:
1. Install Ubuntu VirtualBox (my installation is Ubuntu 16.04 LTS) https://www.lifewire.com/run-ubuntu-within-windows-virtualbox-2202098
    I would dedicate at least 200 GB of hard drive space to this virtualbox.
2. Follow mountainsort instructions for installation on your Ubuntu VirtualBox https://github.com/magland/mountainlab
    
    a. set up your pipeline settings (I copied mine from ms2 and changed some of the parameters to match my default recording configuration)
    
    b. set up your probe files (you can make a csv file from the geometry specified in the kwik prb file). The first column would be inverted y and the second x to make the templates match the expected geometry.
    
    
3. Share folder between host and guest (mine is C://Users/Alan/Desktop/UbuntuShare on the host, and /media/sf_UbuntuShare on the guest)
4. Install guest additions on guest (may have to do this twice).
    command in guest terminal: sudo apt-get install virtualbox-guest-additions-iso
5. Checkout ms4 branch on host.
6. Map the server drive to a drive on your host (mine is mapped to Z:/)
7. Change guest name, username and password in code below:
    (file.write('call %s guestcontrol GUESTNAME run --username USERNAME --password PASSWORD --exe %s -- --text\n' % (pathToVBOXmanage, UbuntuShell)))
7. Change paths in first cell below to reflect your data

In [1]:
## Specify parameters and paths

dataPath = 'Z:/HarveyLab/Tier1/Alan/Data/20171118/p1' ## this code is written to extract a date folder and a penetration folder, if your data is organized differently, you will have to modify
prb = 'poly2' # None # 'poly2' is shorthand for A1x32-poly2-5mm-...etc
UbuntuShareBase = 'C:/Users/Alan/Desktop/UbuntuShare/'
clusteringPath = 'C:/Users/Alan/Documents/Github/clustering-pipelines'
ms4Path = 'C:/Users/Alan/Documents/GitHub/mountainlab/packages/mlpy/mda'

In [3]:
import os
os.chdir(dataPath)
import sys
sys.path.append(clusteringPath)
import read_rhd as rhd
import numpy as np
import glob
import matplotlib.pyplot as plt
from shutil import copy2
import datetime
import subprocess
%matplotlib inline
## import tool to read mda files; mlpy is available on the ms4 github branch (https://github.com/magland/mountainlab/tree/ms4)
sys.path.append(ms4Path)
import mdaio

In [4]:
files = glob.glob('*.rhd')
#for x in files:
#    print(x)
files.sort(key=os.path.getmtime)  ## WARNING: this sorting may be OS sensitive, could use (key=lambda files: files[-10:-4]) instead
print('Sorted Files:')
for x in files:
    print(x)
    
basename = os.path.basename(dataPath) # taking foldername as basename
secondFolder = os.path.basename(os.path.split(dataPath)[0])
print('{0} is basename'.format(basename))
print('{0} files'.format(len(files)))

Sorted Files:
gridIndent_171118_104826.rhd
gridIndent_171118_104852.rhd
gridIndent_171118_104916.rhd
gridIndent_171118_104941.rhd
gridIndent_171118_105006.rhd
gridIndent_171118_105031.rhd
gridIndent_171118_105056.rhd
gridIndent_171118_105121.rhd
gridIndent_171118_105145.rhd
gridIndent_171118_105210.rhd
gridIndent_171118_105235.rhd
gridIndent_171118_105300.rhd
gridIndent_171118_105325.rhd
gridIndent_171118_105350.rhd
gridIndent_171118_105415.rhd
gridIndent_171118_105440.rhd
gridIndent_171118_105505.rhd
gridIndent_171118_105530.rhd
gridIndent_171118_105554.rhd
gridIndent_171118_105619.rhd
gridIndent_171118_105644.rhd
gridIndent_171118_105709.rhd
gridIndent_171118_105734.rhd
gridIndent_171118_105758.rhd
gridIndent_171118_105823.rhd
gridIndent_171118_105848.rhd
gridIndent_171118_105913.rhd
gridIndent_171118_105937.rhd
gridIndent_171118_110002.rhd
gridIndent_171118_110027.rhd
gridIndent_171118_110052.rhd
gridIndent_171118_110117.rhd
gridIndent_171118_110141.rhd
gridIndent_171118_110206.rhd


In [4]:
#batch write .rhd to large raw.mda

numFiles = len(files)

for i, file in enumerate(files):
    print('\n\n\nExtracting file {0} of {1}\n'.format(i+1, numFiles))
    print(file)
    d = rhd.read_rhd(file)
    
    
    if i == 0:
        recordings = d['amplifier_data']
    else:
        recordings = np.concatenate((recordings,d['amplifier_data']),axis=1)
    
    ##save digital input streams as individual files -- different recordings might have different #s of channels
    for digitalChannel in range(d['board_dig_in_data'].shape[0]):
        d['board_dig_in_data'][digitalChannel].tofile(os.path.splitext(file)[0] + 'chan' + str(digitalChannel) + '.di')
    ##save analog input streams as individual files
    for analogChannel in range(d['aux_input_data'].shape[0]):
        d['aux_input_data'][analogChannel].tofile(os.path.splitext(file)[0] + 'chan' + str(analogChannel) + '.ai')


if not os.path.exists(dataPath+'/alldata'):
    os.mkdir(dataPath+'/alldata')
print('\n\nCopying recordings to .mda file')
mdaio.writemda16ui(recordings,dataPath+'/alldata/raw.mda')
print('Finished copying')
del recordings





Extracting file 1 of 135

gridIndent_171118_104826.rhd

Reading Intan Technologies RHD2000 Data File, Version 1.5

Found 32 amplifier channels.
Found 3 auxiliary input channels.
Found 1 supply voltage channel.
Found 0 board ADC channels.
Found 1 board digital input channel.
Found 0 board digital output channels.
Found 0 temperature sensors channels.

File contains 22.032 seconds of data.  Amplifiers were sampled at 20.00 kS/s.

Allocating memory for data...
Reading data from file...
10% done...
20% done...
30% done...
40% done...
50% done...
60% done...
70% done...
80% done...
90% done...
Parsing data...
No missing timestamps in data.
Done!  Elapsed time: 8.7 seconds



Extracting file 2 of 135

gridIndent_171118_104852.rhd

Reading Intan Technologies RHD2000 Data File, Version 1.5

Found 32 amplifier channels.
Found 3 auxiliary input channels.
Found 1 supply voltage channel.
Found 0 board ADC channels.
Found 1 board digital input channel.
Found 0 board digital output channels.
Foun

# copy raw.mda file to UbuntuShare for mountainsort

In [5]:
## Copy data to Ubuntu share

dt = datetime.datetime.now()
secondFolder = os.path.basename(os.path.split(dataPath)[0])
print('containing folder:',secondFolder)
print('basename:',basename)
#yearMonthDay = dt.strftime('%Y%m%d')  
UbuntuSharePath = UbuntuShareBase+secondFolder+'/'+basename+'/'

if not os.path.exists(UbuntuSharePath):
    os.makedirs(UbuntuSharePath)
copy2(dataPath+'/alldata/raw.mda',UbuntuSharePath) ## copies raw.mda to UbuntuShare folder

containing folder: 20171118
basename: p1


'C:/Users/Alan/Desktop/UbuntuShare/20171118/p1/raw.mda'

In [23]:
## define functions for writing and running shell script in Ubuntu virtualbox

def writeUbuntuShell(UbuntuSharePath,basename, probe=None, num_channels=32):
    '''
    UbuntuSharePath is path of path that is shared with Ubuntu Virtualbox
    basename is the string of the containing folder
    probe is shorthand for a probe: 'poly2' for A1x32-Poly2-5mm-50s-177-A32
    
    Returns: string of Ubuntu location of shell
    '''
    oldpath = os.getcwd()
    os.chdir(UbuntuSharePath)
    if probe == 'poly2': # if A1x32-Poly2-5mm-50s-177-A32
        copy2('C://Users/Alan/Documents/Github/clustering-pipelines/poly2Geom.csv',UbuntuSharePath)
        num_channels=32
    dt = datetime.datetime.now()
    secondFolder = os.path.basename(os.path.split(dataPath)[0])
    sortDir = '/home/alan/data/sortProjects/'+secondFolder+'/'+basename # every penetration will have its own sortProject
    #sortDirFull = sortDir+'/datasets/'+basename ## make folder for sortProject with folder name as current date and basename as dataset
    #rawMdaLoc = sortDirFull+'/raw.mda'

    #with open('datasets%s.txt' % (secondFolder) ,'w+') as datasetFile:
    #    datasetFile.write(basename+' /datasets/'+basename+'\n')
    
    #with open('pipelines'+secondFolder+'.txt','w+') as pipelineFile:
    #    pipelineFile.write('ms2alan ms2_alan.pipeline') # default mountainsort pipeline file
    
    with open('params%s.json' % (secondFolder) ,'w+') as paramFile:
        if probe == None:
            paramFile.write('{"samplerate":20000,"sign":-1}') # default sample rate is 20 kHz and usually see negative spikes
        elif probe == 'poly2':
            paramFile.write('{"samplerate":20000,"sign":-1,"adjacency_radius":50}')
    with open('tempShell.sh', 'w+', newline='\n') as shellfile: # newline=\n necessary for linux
        shellfile.write('#! /bin/bash -i\n') # including the -i makes the script interative and this allows for use of default PATH on SSH
        shellfile.write('mkdir -p '+sortDir+'\n')
        
        ## convert .dat to .mda, create prv, and copy to datasets
        shellfile.write('cd /media/sf_UbuntuShare/%s/%s\n' % (secondFolder,basename)) #directory of shared folder in Ubuntu Virtualbox
        #shellfile.write('/home/alan/mountainlab/bin/mdaconvert raw.dat raw.mda --dtype=uint16 --input_format=raw_timeseries --num_channels=%d\n' % (num_channels)) # convert .dat to .mda
        shellfile.write('/home/alan/FImountainlab/bin/prv-create raw.mda\n')
        shellfile.write('cp /media/sf_UbuntuShare/%s/%s/raw.mda.prv %s\n' % (secondFolder,basename,sortDir))
        shellfile.write('cp /home/alan/FImountainlab/packages/mountainsort/pipelines/mountainsort3.mlp %s\n' % (sortDir)) # copy pipeline mlp file to sort directory
        
        ## copy params.json file to sortDir
        
        shellfile.write('cp params%s.json %s/params.json\n' % (secondFolder,sortDir))
        
        if probe == 'poly2':
            shellfile.write('cp poly2Geom.csv %s/geom.csv\n' % (sortDir))

        ## copy datasets.txt and pipelines.txt to sortDir
        #shellfile.write('cp datasets%s.txt %s/datasets.txt\n' % (secondFolder,sortDir))
        #shellfile.write('cp pipelines%s.txt %s/pipelines.txt\n' % (secondFolder,sortDir))
        
        ## move to sortDir and run sorting project
        shellfile.write('cd %s\n' % (sortDir))
        shellfile.write('mlp-run %s/mountainsort3.mlp sort --raw=raw.mda.prv --geom=geom.csv --firings_out=firings.mda --_params=params.json\n'%(sortDir))
    os.chdir(oldpath)
    return '/media/sf_UbuntuShare/%s/%s/tempShell.sh' % (secondFolder, basename)

def runUbuntuShell(UbuntuShell):
    '''
    UbuntuShell is str with linux style directory of shell file
        this directory is returned from the writeUbuntuShell function
    For Alan, that is something like /media/sf_UbuntuShare/DATE/DATABASE/shelltorun.sh
    '''
    with open('ubuntuRun.bat','w+') as file:
        ## change directory to that with share folder
        pathToVBOXmanage = '"C:/Program Files/Oracle/VirtualBox/VBoxManage.exe"'
        file.write('title run ubuntu shell\n')
        
        #####
        ## I've commented out the following bit of code because I set my vbox to always start upon login to Windows
        ## see https://hellojason.net/blog/automatically-launch-a-headless-virtualbox-vm-at-bootup/
        ## You can uncomment to auto startup your virtualbox with this script
        ####
        
        ## start virtual box
        #file.write('call %s startvm "Ubuntu"\n' %(pathToVBOXmanage))
        ## i set up the virtualbox to set the RDONLYHOST property on startup
        ## this wait command waits for that property to change before proceeding
        #file.write('call %s guestproperty wait "Ubuntu" RDONLYHOST\n' %(pathToVBOXmanage))
        
        
        ## executes the script UbuntuShell
        # change username and password to reflect your virtualbox login
        file.write('call %s guestcontrol "Ubuntu" run --username alan --password 123ubuntubox --exe %s -- --text\n' % (pathToVBOXmanage, UbuntuShell))
        
    p = subprocess.Popen('ubuntuRun.bat',shell=True) #run the BAT file.
    stdout, stderr = p.communicate()
    # in jupyter, the stdout appears in the cmd window rather than the browser
    return stdout, stderr

In [24]:
UbuntuSharePath = UbuntuShareBase+secondFolder+'/'+basename+'/'
ubuntuLocation = writeUbuntuShell(UbuntuSharePath,basename,probe=prb)

In [28]:
stdout, stderr = runUbuntuShell(ubuntuLocation)